In [ ]:
import cv2, os
import numpy as np
import trimesh
from tqdm import tqdm
from PIL import Image
from skimage.transform import resize

In [ ]:
path = "EHF/"

data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.jpg'):
        image_path = path + images
        image = cv2.imread(image_path)
        image = cv2.resize(image,(256,256))
        data.append(image)

In [ ]:
def load_ply_data(ply_file_path):
    mesh = trimesh.load_mesh(ply_file_path)
    return mesh.vertices, mesh.faces

In [ ]:
faces_data = []
vertices_data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.ply'):
        ply_path = path + images
        vertices, faces = load_ply_data(ply_path)
        faces_data.append(faces)
        vertices_data.append(vertices)

In [ ]:
data = np.array(data)
faces_data = np.array(faces_data)
vertices_data = np.array(vertices_data)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
num_vertices = 10475
num_faces = 20908

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
vgg_output = vgg16.layers[-1].output

conv1 = Conv2D(64, kernel_size=(3, 3), activation='relu')(vgg_output)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(conv1)
flatten = Flatten()(conv2)
dense1 = Dense(128, activation='relu')(flatten)

reshape_vertices = Dense(num_vertices * 3, activation='linear')(dense1)
reshape_faces = Dense(num_faces * 3, activation='linear')(dense1)

reshape_vertices = Reshape((num_vertices, 3))(reshape_vertices)
reshape_faces = Reshape((num_faces, 3))(reshape_faces)

# Create the model
model = Model(inputs=vgg16.input, outputs=[reshape_vertices, reshape_faces])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(data, [vertices_data, faces_data], batch_size=2, epochs=983)

In [ ]:
model.save("3dcheck.h5")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("3dcheck.h5")

In [ ]:
image = cv2.imread("1.jpg")
image = cv2.resize(image,(256,256))
image1 = np.expand_dims(image,axis=0)
predictions = model.predict(image1)
print(predictions[0].shape, predictions[1].shape)

In [ ]:
predicted_vertices = predictions[0].squeeze(axis=0)
predicted_faces = predictions[1].squeeze(axis=0)

In [ ]:
predicted_vertices = np.reshape(predicted_vertices, (-1, 3))
predicted_faces = np.reshape(predicted_faces, (-1, 3))

In [ ]:
predicted_vertices.shape, predicted_faces.shape

In [ ]:
# Check and adjust the vertex indices in the faces array
#max_vertex_index = predicted_vertices.shape[0] - 1
#predicted_faces = np.where(predicted_faces > max_vertex_index, max_vertex_index, predicted_faces)

# Create a trimesh object using the predicted vertices and faces
mesh = trimesh.Trimesh(vertices=predicted_vertices, faces=predicted_faces)

# Save the mesh as a .ply file
output_file = "predicted_mesh.ply"
mesh.export(output_file)